In [2]:
# 📘 PRIMERA – Inference & ROUGE Evaluation (Steps 1–3)

from google.colab import files
print("📁 Please upload your `multidoc_test.jsonl` file...")
uploaded = files.upload()


📁 Please upload your `multidoc_test.jsonl` file...


Saving multidoc_test.jsonl to multidoc_test.jsonl


In [3]:
!pip install transformers rouge-score > /dev/null

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "allenai/PRIMERA"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/20.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/283 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.79G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/197 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.79G [00:00<?, ?B/s]

In [5]:
import json

with open("/content/multidoc_test.jsonl", "r") as f:
    test_data = [json.loads(line) for line in f]

# We'll test on 10 samples only for now
test_data = test_data[:10]

generated_summaries = []
for item in test_data:
    input_text = " </s> ".join(item["documents"])
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=4096)
    outputs = model.generate(
        **inputs,
        num_beams=4,
        max_length=256,
        min_length=32,
        length_penalty=2.0,
        early_stopping=True,
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_summaries.append({
        "generated": decoded,
        "reference": item["summary"]
    })

print(f"✅ Done! Generated {len(generated_summaries)} summaries.")


Input ids are automatically padded from 2084 to 2560 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 1237 to 1536 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 1795 to 2048 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 906 to 1024 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 2111 to 2560 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 3581 to 3584 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 2757 to 3072 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 2554 to 2560 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 2073 to 2560 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 2088 to 2560 to be a mult

✅ Done! Generated 10 summaries.


In [6]:
# ✅ Step 3: ROUGE Evaluation with Timing
from rouge_score import rouge_scorer
import time

scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

scores = {"rouge1": [], "rouge2": [], "rougeL": []}
start_time = time.time()

for pair in generated_summaries:
    score = scorer.score(pair["reference"], pair["generated"])
    for k in scores:
        scores[k].append(score[k].fmeasure)

end_time = time.time()
avg_scores = {k: sum(v)/len(v) for k, v in scores.items()}
inference_time = end_time - start_time

# Print ROUGE Scores
print("\n🔍 ROUGE Scores:")
for k, v in avg_scores.items():
    print(f"{k.upper()}: {v:.4f}")

# Print Inference Time
print(f"\n⏱️ Total inference + evaluation time (seconds): {inference_time:.2f}")
print(f"🧮 Avg time per sample: {inference_time / len(generated_summaries):.2f} sec")




🔍 ROUGE Scores:
ROUGE1: 0.3062
ROUGE2: 0.0898
ROUGEL: 0.1744

⏱️ Total inference + evaluation time (seconds): 0.16
🧮 Avg time per sample: 0.02 sec
